In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# 評価指標
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [98]:
from sklearn.linear_model import LinearRegression, SGDRegressor, HuberRegressor
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import PassiveAggressiveRegressor, ARDRegression


In [2]:
with open('train.csv') as f:
    df = pd.read_csv(f)
df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [39]:
X = df[["GrLivArea", "YearBuilt"]].values
y = df["SalePrice"].values

In [54]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.8, random_state=None)
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("Y_train:", Y_train.shape)
print("Y_test:", Y_test.shape)

X_train: (1168, 2)
X_test: (292, 2)
Y_train: (1168,)
Y_test: (292,)


In [88]:
# Ｋ個に分割
x = np.arange(10)
kf = KFold(n_splits=5, shuffle=False)
for i in kf.split(x):
    print(i)

(array([2, 3, 4, 5, 6, 7, 8, 9]), array([0, 1]))
(array([0, 1, 4, 5, 6, 7, 8, 9]), array([2, 3]))
(array([0, 1, 2, 3, 6, 7, 8, 9]), array([4, 5]))
(array([0, 1, 2, 3, 4, 5, 8, 9]), array([6, 7]))
(array([0, 1, 2, 3, 4, 5, 6, 7]), array([8, 9]))


# 問題1　ブレンディングのスクラッチ実装

## 1つ目　複数モデル

In [56]:
# 1つのモデルでトレーニング、推定を行う

# 単一モデル
model = [LinearRegression(),
         SVR(),
         DecisionTreeRegressor()]


# Cross-validation
kf = KFold(n_splits=5,random_state=None, shuffle=False)

# 5つのパートで検証されたメトリクスMSE
for regr in model:
    result = -cross_val_score(regr,X,y, cv = kf, scoring = "neg_mean_squared_error")
    result_mean = np.mean(result)
    
    # print('CV_MSE:',result)
    print('CV_MSE_MEAN:{:.3g}'.format(result_mean),'MODEL:',str(regr))

CV_MSE_MEAN:2.22e+09 MODEL: LinearRegression()
CV_MSE_MEAN:6.61e+09 MODEL: SVR()
CV_MSE_MEAN:2.79e+09 MODEL: DecisionTreeRegressor()


In [61]:
# 各モデルで予測を行う
reg1 = LinearRegression().fit(X_train,Y_train)
reg2 = SVR().fit(X_train,Y_train)
reg3 = DecisionTreeRegressor().fit(X_train,Y_train)

pred1 = reg1.predict(X_test)
pred2 = reg2.predict(X_test)
pred3 = reg3.predict(X_test)

# ブレンディング
pred_brend = np.mean([pred1, pred2, pred3], axis=0)

# 評価
mse_brend = mean_squared_error(Y_test, pred_brend)

print('ブレンディング1のMSE：{:.3g}'.format(mse_brend))

ブレンディング1のMSE：2.16e+09


## 2つ目　単一モデルパラメータ変更

In [64]:
# Cross-validation
kf = KFold(n_splits=5,random_state=None, shuffle=False)

params = [1,10,20]

# 5つのパートで検証されたメトリクスMSE
for rn in params:
    # 単一モデル
    regr = DecisionTreeRegressor(random_state=rn)
    result = -cross_val_score(regr,X,y, cv = kf, scoring = "neg_mean_squared_error")
    result_mean = np.mean(result)
    
    # print('CV_MSE:',result)
    print('CV_MSE_MEAN:{:.3g}'.format(result_mean),'MODEL:',str(regr))

CV_MSE_MEAN:2.9e+09 MODEL: DecisionTreeRegressor(random_state=1)
CV_MSE_MEAN:2.8e+09 MODEL: DecisionTreeRegressor(random_state=10)
CV_MSE_MEAN:2.9e+09 MODEL: DecisionTreeRegressor(random_state=20)


In [63]:
# 各モデルで予測を行う
reg1 = DecisionTreeRegressor(random_state=1).fit(X_train,Y_train)
reg2 = DecisionTreeRegressor(random_state=10).fit(X_train,Y_train)
reg3 = DecisionTreeRegressor(random_state=20).fit(X_train,Y_train)

pred1 = reg1.predict(X_test)
pred2 = reg2.predict(X_test)
pred3 = reg3.predict(X_test)

# ブレンディング
pred_brend = np.mean([pred1, pred2, pred3], axis=0)

# 評価
mse_brend = mean_squared_error(Y_test, pred_brend)

print('ブレンディング2のMSE：{:.3g}'.format(mse_brend))

ブレンディング2のMSE：2.54e+09


## 3つ目　説明変数の標準化

In [65]:
# データの標準化
X_std_data = StandardScaler().fit_transform(df[["GrLivArea", "YearBuilt"]])

In [78]:
X_train, X_test, Y_train, Y_test = train_test_split(X_std_data, y, train_size=0.8, random_state=None)
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("Y_train:", Y_train.shape)
print("Y_test:", Y_test.shape)

X_train: (1168, 2)
X_test: (292, 2)
Y_train: (1168,)
Y_test: (292,)


In [79]:
# 1つのモデルでトレーニング、推定を行う

# 単一モデル
model = [LinearRegression(),
         SVR(),
         DecisionTreeRegressor()]


# Cross-validation
kf = KFold(n_splits=5,random_state=None, shuffle=False)

# 5つのパートで検証されたメトリクスMSE
for regr in model:
    result = -cross_val_score(regr,X,y, cv = kf, scoring = "neg_mean_squared_error")
    result_mean = np.mean(result)
    
    # print('CV_MSE:',result)
    print('CV_MSE_MEAN:{:.3g}'.format(result_mean),'MODEL:',str(regr))

CV_MSE_MEAN:2.22e+09 MODEL: LinearRegression()
CV_MSE_MEAN:6.61e+09 MODEL: SVR()
CV_MSE_MEAN:2.87e+09 MODEL: DecisionTreeRegressor()


In [80]:
# 各モデルで予測を行う
reg1 = LinearRegression().fit(X_train,Y_train)
reg2 = SVR().fit(X_train,Y_train)
reg3 = DecisionTreeRegressor().fit(X_train,Y_train)

pred1 = reg1.predict(X_test)
pred2 = reg2.predict(X_test)
pred3 = reg3.predict(X_test)

# ブレンディング
pred_brend = np.mean([pred1, pred2, pred3], axis=0)

# 評価
mse_brend = mean_squared_error(Y_test, pred_brend)

print('ブレンディング1のMSE：{:.3g}'.format(mse_brend))

ブレンディング1のMSE：2.15e+09


# 問題2　バギングのスクラッチ実装

In [81]:
X_train, X_test, Y_train, Y_test = train_test_split(X, y, train_size=0.8, random_state=None)
print("X_train:", X_train.shape)
print("X_test:", X_test.shape)
print("Y_train:", Y_train.shape)
print("Y_test:", Y_test.shape)

X_train: (1168, 2)
X_test: (292, 2)
Y_train: (1168,)
Y_test: (292,)


In [83]:
# 単一モデル
model = DecisionTreeRegressor().fit(X_train, Y_train)
y_pred = model.predict(X_test)

# 評価
mse = mean_squared_error(Y_test, y_pred)
print("MSE:{:.3g}".format(mse))

MSE:3.04e+09


In [85]:
# バギング
n = 20
models = []

for i in range(n):
    X_bagging, X_, y_bagging, y_ = train_test_split(X_train, Y_train, train_size=0.2, shuffle=True)
    
    model = DecisionTreeRegressor().fit(X_bagging, y_bagging)
    models.append(model)
    
y_pred = np.zeros(len(X_test))

for reg in models:
    pred = reg.predict(X_test)
    y_pred += pred
    
y_pred = y_pred / n

# 評価
mse = mean_squared_error(Y_test, y_pred)
print("MSE : {:.3g}".format(mse))

MSE : 1.81e+09


# 問題3　スタッキングのスクラッチ実装

In [107]:
class Stacking():
    def __init__(self, max_depth, splits, models):
        self.max_depth = max_depth
        self.n_splits = splits
        self.models = models
        self.fit_models = []
        
    def blending(self, X, y, m):
        self.y_blend = np.zeros(len(X))
        
        kf = KFold(n_splits=self.n_splits, shuffle=False)
        
        for train_index, valid_index in kf.split(X):
            X_train, X_valid = X[train_index], X[valid_index]
            y_train, y_valid = y[train_index], y[valid_index]
            
            y_train = y_train.ravel()
            y_valid = y_valid.ravel()
            
            regr = m
            regr.fit(X_train, y_train)
            self.fit_models.append(regr)
            
            self.y_blend[valid_index] = regr.predict(X_valid)
            
    def fit(self, X, y, depth):
        self.depth = depth
        
        if self.depth == self.max_depth:
            self.model = self.models[self.depth]
            self.model.fit(X, y)
            return
        
        models = self.models[self.depth]
        self.y_blending = np.zeros([len(X), len(models)])
        
        for i, mdl in enumerate(models):
            self.blending(X, y, mdl)
            self.y_blending[:, i] = self.y_blend
        blend_y = self.y_blending
        
        # 再帰的
        self.bld = Stacking(self.max_depth, self.n_splits, self.models)
        self.bld.fit(blend_y, y, depth+1)
        
    def predict(self, X):
        if self.depth == self.max_depth:
            y_pred = self.model.predict(X)
            return y_pred
        else:
            tmp = 0
            self.y_pred = np.zeros(len(X))
            self.y_next = np.zeros([len(X), len(self.models[self.depth])])
            
            for mdl in self.fit_models:
                tmp+=1
                self.y_pred += mdl.predict(X)
                
                if tmp%self.n_splits == 0:
                    self.y_pred = self.y_pred/self.n_splits
                    self.y_next[:, int(tmp/self.n_splits)-1] = self.y_pred
                    self.y_pred = np.zeros(len(X))
                    
            y_pred = self.bld.predict(self.y_next)
        return y_pred

In [127]:
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size=0.8,random_state=None)

print('X_train:',X_train.shape)
print('y_train:',y_train.shape)
print('X_valid:',X_valid.shape)
print('y_valid:',y_valid.shape)

X_train: (1168, 2)
y_train: (1168,)
X_valid: (292, 2)
y_valid: (292,)


In [128]:
model = {0:[LinearRegression(),DecisionTreeRegressor(),RandomForestRegressor()],
         1:[ARDRegression(),SGDRegressor(),DecisionTreeRegressor()],
         2:[HuberRegressor(),ARDRegression(),RandomForestRegressor()],
         3:LinearRegression()
         }

stk = Stacking(max_depth=3,splits=5,models=model)
stk.fit(X_train,y_train,0)
y_pred = stk.predict(X_valid)
print(y_pred)

# 評価
mse = mean_squared_error(y_valid,y_pred)
print('MSE : {:.3g}'.format(mse))

[190673.45506051  90538.59818542 208450.92014096 180213.36809457
 149876.07215062  90001.79998781 147394.97679882 151120.34122798
 128515.47748179 219545.67111958 111776.88006304 179676.77562979
 217956.2231433  157305.75967155 125622.74579149 199323.10760917
 186093.7710248  129351.15661185 222363.58787941 144058.75873607
 217010.56586303 205119.6955887  163099.53934843 231138.7728962
 203499.31689174 211812.23862182 188437.27598641 133054.48921531
 144461.62002613 227417.85834039 132342.72293035 219750.87177416
 109373.98450418 214293.05927775 288773.53595286 220895.13340295
 154663.7003625  207779.30230797 129536.60176451 233291.30313395
 228475.88167847 145017.79357249 226234.82810288 101316.09870503
 197710.82769753 122888.10232408 216628.1738951  167405.78614388
 145394.63171054 140564.60215003 142765.78353315 175573.21994614
 187335.01381841 213536.51936922 152149.26475259 197200.81024624
 193485.19869272 131255.81512183 143550.4205202  184564.0131524
 131030.18378426 209769.755